<a href="https://colab.research.google.com/github/RRM2021/USAID-MODELS/blob/main/REPORTES_BASE_DE_DATOS_RIS_07_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
    import pyspark
    print("PySpark is already installed.")
except ImportError:
    print("PySpark is not installed. Installing now...")
    !pip install pyspark
    print("PySpark installation complete.")


PySpark is not installed. Installing now...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=4e61e84a171ad0bd1608d6c30165ddcb2a7f81eb8ce2bed6ce796d2f33846389
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark
PySpark installation complete.


In [ ]:
try:
    from pyspark import SparkContext, SparkConf
    from pyspark.sql import SparkSession
except ImportError as e:
    printmd('<<<<<!!!!! Please restart your kernel after installing Apache Spark !!!!!>>>>>')

In [ ]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

spark = SparkSession \
    .builder \
    .getOrCreate()

In [ ]:
import gspread
from google.colab import auth

In [ ]:
!pip install --upgrade google-api-python-client oauth2client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 82.3 MB/s eta 0:00:00
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.137.0
    Uninstalling google-api-python-client-2.137.0:
      Successfully uninstalled google-api-python-client-2.137.0


In [ ]:
auth.authenticate_user()

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
spreadsheet = gc.open('Tablas de Base de Datos RIS_1_07_2024')
worksheet = spreadsheet.worksheet('OIRs')

# Retrieve all values from the worksheet
data_1 = worksheet.get_all_values()

# Extract column titles from the first row
column_titles_1 = data_1[0]

# Extract data rows excluding the first row
data_rows_1 = data_1[1:]

# Create DataFrame with column titles
data_frame_1 = spark.createDataFrame(data_rows_1, column_titles_1)

data_frame_1.createOrReplaceTempView("OIR")

# Show DataFrame
data_frame_1.show()

+---------+--------+--------------------+--------------------+-----------+
|ID_REGION|  ID_OIR|          NOMBRE_OIR|     DESCRIPCIÓN_OIR|OIR_VIGENTE|
+---------+--------+--------------------+--------------------+-----------+
|      PCF|PCF_OIR1|Fortalecimiento d...|Contribuir a la c...|         SÍ|
|      PCF|PCF_OIR2| Inclusión económica|Inclusión económi...|         SÍ|
|      PCF|PCF_OIR3|Acceso a bienes y...|Fortalecer las ca...|         SÍ|
|      PCF|PCF_OIR4|Entornos protecto...|Las actividades d...|         SÍ|
|      BCN|BCN_OIR1|Fortalecimiento y...|Formación a produ...|         SÍ|
|      BCN|BCN_OIR2|Fortalecimiento d...|Intercambio de co...|         NO|
|      BCN|BCN_OIR3|Fortalecer la gob...|Expansión de serv...|         SÍ|
|      BCN|BCN_OIR4|Enfoque diferenci...|Fortalecer las ca...|         SÍ|
|      CAT|CAT_OIR1|Ciudadanía empode...|Fortalecimiento d...|         SÍ|
|      CAT|CAT_OIR2|Gobernanza y gobe...|Fortalecimiento d...|         SÍ|
|      CAT|CAT_OIR3|Servi

In [ ]:
# Filter OIR where OIR_VIGENTE = SÍ
data_frame = spark.sql("SELECT * FROM OIR WHERE OIR_VIGENTE = 'SÍ'")
data_frame.show()

+---------+--------+--------------------+--------------------+-----------+
|ID_REGION|  ID_OIR|          NOMBRE_OIR|     DESCRIPCIÓN_OIR|OIR_VIGENTE|
+---------+--------+--------------------+--------------------+-----------+
|      PCF|PCF_OIR1|Fortalecimiento d...|Contribuir a la c...|         SÍ|
|      PCF|PCF_OIR2| Inclusión económica|Inclusión económi...|         SÍ|
|      PCF|PCF_OIR3|Acceso a bienes y...|Fortalecer las ca...|         SÍ|
|      PCF|PCF_OIR4|Entornos protecto...|Las actividades d...|         SÍ|
|      BCN|BCN_OIR1|Fortalecimiento y...|Formación a produ...|         SÍ|
|      BCN|BCN_OIR3|Fortalecer la gob...|Expansión de serv...|         SÍ|
|      BCN|BCN_OIR4|Enfoque diferenci...|Fortalecer las ca...|         SÍ|
|      CAT|CAT_OIR1|Ciudadanía empode...|Fortalecimiento d...|         SÍ|
|      CAT|CAT_OIR2|Gobernanza y gobe...|Fortalecimiento d...|         SÍ|
|      CAT|CAT_OIR3|Servicios e infra...|Dinamizar y mejor...|         SÍ|
|      CAT|CAT_OIR4|Forta

In [ ]:
# Filter OIR where OIR_VIGENTE = SÍ
data_frame = spark.sql("SELECT * FROM OIR WHERE OIR_VIGENTE = 'SÍ' AND (ID_REGION = 'PCF' OR ID_REGION = 'BCN')")
data_frame.show()

+---------+--------+--------------------+--------------------+-----------+
|ID_REGION|  ID_OIR|          NOMBRE_OIR|     DESCRIPCIÓN_OIR|OIR_VIGENTE|
+---------+--------+--------------------+--------------------+-----------+
|      PCF|PCF_OIR1|Fortalecimiento d...|Contribuir a la c...|         SÍ|
|      PCF|PCF_OIR2| Inclusión económica|Inclusión económi...|         SÍ|
|      PCF|PCF_OIR3|Acceso a bienes y...|Fortalecer las ca...|         SÍ|
|      PCF|PCF_OIR4|Entornos protecto...|Las actividades d...|         SÍ|
|      BCN|BCN_OIR1|Fortalecimiento y...|Formación a produ...|         SÍ|
|      BCN|BCN_OIR3|Fortalecer la gob...|Expansión de serv...|         SÍ|
|      BCN|BCN_OIR4|Enfoque diferenci...|Fortalecer las ca...|         SÍ|
+---------+--------+--------------------+--------------------+-----------+



In [ ]:
import pandas as pd
from google.colab import files

# Convert PySpark DataFrame to Pandas DataFrame
df = data_frame.toPandas()

In [ ]:
# Count unique values in 'SubCategory' and group by 'Category'
grouped_df = df.groupby('ID_REGION')['ID_OIR'].nunique().reset_index()
grouped_df.columns = ['ID_REGION', 'UniqueID_OIR']

print(grouped_df)


  ID_REGION  UniqueID_OIR
0       BCN             3
1       BOG             3
2       CAT             4
3       CCA             2
4       CQT             2
5       PCF             4
6       SDC             2


In [ ]:
try:
    import plotly
    print("plotly openpyxl is already installed.")
except ImportError:
    print("plotly openpyxl is not installed. Installing now...")
    !pip install plotly openpyxl
    print("plotly openpyxl installation complete.")

plotly openpyxl is already installed.


In [ ]:
import pandas as pd
import plotly.express as px

# Sort the DataFrame by 'UniqueID_INICIATIVAS' in descending order
grouped_df_sorted = grouped_df.sort_values(by='UniqueID_OIR', ascending=False)

# Create the bar plot
fig = px.bar(grouped_df_sorted, x='ID_REGION', y='UniqueID_OIR', title='OPORTUNIDADES DE INTEGRACIÓN REGIONAL OIR POR REGION')

# Customize the plot
fig.update_layout(
    xaxis_title='REGIÓN',
    yaxis_title='NÚMERO DE OIR',
    title={
        'text': 'OPORTUNIDADES DE INTEGRACIÓN REGIONAL OIR POR REGION',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    font=dict(
        family="Arial",
        size=15,
        color="black"
    ),
    yaxis=dict(
        tickformat=',d',
        dtick=1
    )
)

# Update the text position to be inside the bars
fig.update_traces(texttemplate='%{y}', textposition='inside')

# Show the plot
fig.show()

In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Filter the DataFrame where ID_REGION is 'PCF' or 'BCN'
filtered_df = df[df['ID_REGION'].isin(['PCF', 'BCN'])]

# Select only the ID_REGION and NOMBRE_OIR columns
filtered_df = filtered_df[['ID_REGION', 'NOMBRE_OIR']]

# Create the table
fig = go.Figure(data=[go.Table(
    columnwidth=[50, 150],  # Set the width for each column
    header=dict(values=list(filtered_df.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[filtered_df[col] for col in filtered_df.columns],
               fill_color='lavender',
               align='left'))
])

# Customize the layout
fig.update_layout(
    title={
        'text': 'OIRs EN PACÍFICO Y FRONTERA NARIÑENSE Y BAJO CAUCA Y NORDESTE ANTIOQUEÑO',
        'font': {'size': 12},  # Reduce the title font size
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'pad': {'b': 10}  # Reduce the padding (distance) to the table
    },
    width=910,  # Adjust the overall width of the table
    height=410  # Reduce the overall height of the canvas
)

# Show the plot
fig.show()


In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Filter the DataFrame where ID_REGION is 'PCF' or 'BCN'
filtered_df = df[df['ID_REGION'].isin(['PCF', 'BCN', 'CAT', 'BOG', 'SDC', 'CQT', 'CCA'])]

# Select only the ID_REGION and NOMBRE_OIR columns
filtered_df = filtered_df[['ID_REGION', 'NOMBRE_OIR']]

fig = px.treemap(filtered_df, path=['ID_REGION', 'NOMBRE_OIR'], title='OIRs EN REGIONES')

# Customize the layout
fig.update_layout(
    title_x=0.5,  # Center the title
)

fig.show()


In [ ]:
import pandas as pd
import plotly.express as px

# Create a crosstab for the heatmap
crosstab_df = pd.crosstab(filtered_df['NOMBRE_OIR'], filtered_df['ID_REGION'])

# Plot the heatmap
fig = px.imshow(crosstab_df, labels=dict(x="Región", y="Nombre de las OIR", color="Count"),
                title="OIR por Región")

# Customize the layout, increase width to fit all ID_OIRs and remove the color bar
fig.update_layout(
    title_x=0.5,  # Center the title
    width=1400,   # Increase the width of the plot
    height=800,   # Adjust the height if needed
    coloraxis_showscale=False,  # Remove the color bar
    xaxis=dict(showgrid=False),  # Remove gridlines from x-axis
    yaxis=dict(showgrid=False)   # Remove gridlines from y-axis
)

# Add gridlines between the rows and columns
fig.update_traces(xgap=1, ygap=1)

fig.show()

In [ ]:
# Create a crosstab for the heatmap
crosstab_df = pd.crosstab(filtered_df['NOMBRE_OIR'], filtered_df['ID_REGION'])

# Plot the heatmap
fig = px.imshow(crosstab_df, labels=dict(x="Región", y="Nombre OIR", color="Count"),
                title="OIR por Región")

# Customize the layout
fig.update_layout(
    title_x=0.5  # Center the title
)

fig.show()



In [ ]:
fig = px.scatter(filtered_df, x='ID_REGION', y='NOMBRE_OIR', color='ID_REGION',
                 title='OIRs por Region')

# Customize the layout
fig.update_layout(
    title_x=0.5  # Center the title
)

fig.show()


In [ ]:
fig = px.sunburst(filtered_df, path=['ID_REGION', 'NOMBRE_OIR'], title='Sunburst Chart of OIR Distribution')

# Customize the layout
fig.update_layout(
    title_x=0.5  # Center the title
)

fig.show()


In [ ]:
#spark.sql("SELECT COUNT(*) FROM IE_RIS")
# Write and execute the SQL query
result = spark.sql("SELECT COUNT(DISTINCT ID_OIR) AS numero_OIR FROM OIR")

# Show the result
result.show()

+----------+
|numero_OIR|
+----------+
|        21|
+----------+



ACTIVIDADES EN OIR

In [ ]:
spreadsheet = gc.open('Tablas de Base de Datos RIS_1_07_2024')
worksheet = spreadsheet.worksheet('OIR_ACTIVIDAD')

# Retrieve all values from the worksheet
data_1 = worksheet.get_all_values()

# Extract column titles from the first row
column_titles_1 = data_1[0]

# Extract data rows excluding the first row
data_rows_1 = data_1[1:]

# Create DataFrame with column titles
data_frame_1 = spark.createDataFrame(data_rows_1, column_titles_1)

data_frame_1.createOrReplaceTempView("ACTIVIDAD")

# Show DataFrame
data_frame_1.show()

+---------+--------+--------------------+------------+--------------------+-------+
|ID_REGION|  ID_OIR|          NOMBRE_OIR|ID_ACTIVIDAD|    NOMBRE_ACTIVIDAD|VIGENTE|
+---------+--------+--------------------+------------+--------------------+-------+
|      PCF|PCF_OIR1|Fortalecimiento d...|          RC|Somos Comunidad -...|     SI|
|      PCF|PCF_OIR1|Fortalecimiento d...|         OTI|Colombia Transfor...|     SI|
|      PCF|PCF_OIR1|Fortalecimiento d...|         GEP|Generando Equidad...|     SI|
|      PCF|PCF_OIR1|Fortalecimiento d...|         YRA|Programa de Jóven...|     SI|
|      PCF|PCF_OIR1|Fortalecimiento d...|          JI|Programa Justicia...|     SI|
|      PCF|PCF_OIR1|Fortalecimiento d...|       IPACE|     Juntanza Étnica|     SI|
|      PCF|PCF_OIR1|Fortalecimiento d...|         ROF|Restaurando Nuest...|     SI|
|      PCF|PCF_OIR1|Fortalecimiento d...|  INSPIRAPAZ|          Inspirapaz|     SI|
|      PCF|PCF_OIR1|Fortalecimiento d...|          RG|Gobernabilidad Re...| 

In [ ]:
# Filter OIR where VIGENTE = SÍ
data_frame_1 = spark.sql("SELECT * FROM ACTIVIDAD WHERE VIGENTE = 'SI'")
data_frame_1.show()

+---------+--------+--------------------+------------+--------------------+-------+
|ID_REGION|  ID_OIR|          NOMBRE_OIR|ID_ACTIVIDAD|    NOMBRE_ACTIVIDAD|VIGENTE|
+---------+--------+--------------------+------------+--------------------+-------+
|      PCF|PCF_OIR1|Fortalecimiento d...|          RC|Somos Comunidad -...|     SI|
|      PCF|PCF_OIR1|Fortalecimiento d...|         OTI|Colombia Transfor...|     SI|
|      PCF|PCF_OIR1|Fortalecimiento d...|         GEP|Generando Equidad...|     SI|
|      PCF|PCF_OIR1|Fortalecimiento d...|         YRA|Programa de Jóven...|     SI|
|      PCF|PCF_OIR1|Fortalecimiento d...|          JI|Programa Justicia...|     SI|
|      PCF|PCF_OIR1|Fortalecimiento d...|       IPACE|     Juntanza Étnica|     SI|
|      PCF|PCF_OIR1|Fortalecimiento d...|         ROF|Restaurando Nuest...|     SI|
|      PCF|PCF_OIR1|Fortalecimiento d...|  INSPIRAPAZ|          Inspirapaz|     SI|
|      PCF|PCF_OIR1|Fortalecimiento d...|          RG|Gobernabilidad Re...| 

In [ ]:
# Filter OIR where VIGENTE = SÍ and ID_REGION is either 'PCF' or 'BCN'
data_frame_1 = spark.sql("SELECT * FROM ACTIVIDAD WHERE VIGENTE = 'SI' AND (ID_REGION = 'PCF' OR ID_REGION = 'BCN')")
data_frame_1.show()

+---------+--------+--------------------+------------+--------------------+-------+
|ID_REGION|  ID_OIR|          NOMBRE_OIR|ID_ACTIVIDAD|    NOMBRE_ACTIVIDAD|VIGENTE|
+---------+--------+--------------------+------------+--------------------+-------+
|      PCF|PCF_OIR1|Fortalecimiento d...|          RC|     Somos Comunidad|     SI|
|      PCF|PCF_OIR1|Fortalecimiento d...|         OTI|Colombia Transfor...|     SI|
|      PCF|PCF_OIR1|Fortalecimiento d...|         GEP|Generando Equidad...|     SI|
|      PCF|PCF_OIR1|Fortalecimiento d...|         YRA|Jóvenes Resilient...|     SI|
|      PCF|PCF_OIR1|Fortalecimiento d...|          JI| Justicia Inclusiva |     SI|
|      PCF|PCF_OIR1|Fortalecimiento d...|       IPACE|     Juntanza Étnica|     SI|
|      PCF|PCF_OIR1|Fortalecimiento d...|         ROF|Restaurando Nuest...|     SI|
|      PCF|PCF_OIR1|Fortalecimiento d...|INSPIRA PAZ |         Inspira Paz|     SI|
|      PCF|PCF_OIR1|Fortalecimiento d...|          RG|Gobernabilidad Re...| 

In [ ]:
import pandas as pd
from google.colab import files

# Convert PySpark DataFrame to Pandas DataFrame
df_actividad = data_frame_1.toPandas()

In [ ]:
# Count unique values in 'SubCategory' and group by 'Category'
grouped_df_act = df_actividad.groupby('ID_OIR')['ID_ACTIVIDAD'].nunique().reset_index()
grouped_df_act.columns = ['ID_OIR', 'UniqueID_ACTIVIDAD']

print(grouped_df_act)


      ID_OIR  UniqueID_ACTIVIDAD
0   BCN_OIR1                   8
1   BCN_OIR3                  12
2   BCN_OIR4                  11
3   BOG_OIR1                   8
4   BOG_OIR2                   5
5   BOG_OIR3                   8
6   CAT_OIR1                   9
7   CAT_OIR2                   7
8   CAT_OIR3                   3
9   CAT_OIR4                   6
10  CCA_OIR1                   8
11  CCA_OIR2                   4
12  CQG_OIR1                  11
13  CQG_OIR2                  12
14  PCF_OIR1                   9
15  PCF_OIR2                   8
16  PCF_OIR3                   9
17  PCF_OIR4                   5
18  SDC_OIR1                   4
19  SDC_OIR2                   4


In [ ]:
import pandas as pd
import plotly.express as px

# Create a crosstab for the heatmap
crosstab_df = pd.crosstab(df_actividad['NOMBRE_ACTIVIDAD'], df_actividad['ID_OIR'])

# Plot the heatmap
fig = px.imshow(crosstab_df, labels=dict(x="OIR", y="Actividades de USAID", color="Count"),
                title="Actividades de USAID en las OIR")

# Customize the layout, increase width to fit all ID_OIRs and remove the color bar
fig.update_layout(
    title_x=0.5,  # Center the title
    width=1400,   # Increase the width of the plot
    height=1000,   # Adjust the height if needed
    coloraxis_showscale=False,  # Remove the color bar
    xaxis=dict(showgrid=False),  # Remove gridlines from x-axis
    yaxis=dict(showgrid=False)   # Remove gridlines from y-axis
)

# Add gridlines between the rows and columns
fig.update_traces(xgap=1, ygap=1)

fig.show()




In [ ]:
import pandas as pd
import plotly.express as px

# Sort the DataFrame by 'UniqueID_INICIATIVAS' in descending order
grouped_df_sorted = grouped_df_act.sort_values(by='UniqueID_ACTIVIDAD', ascending=False)

# Create the bar plot
fig = px.bar(grouped_df_sorted, x='ID_OIR', y='UniqueID_ACTIVIDAD', title='ACTIVIDADES DE USAID POR OIR')

# Customize the plot
fig.update_layout(
    xaxis_title='OIR',
    yaxis_title='NÚMERO DE ACTIVIDADES',
    title={
        'text': 'NÚMERO DE ACTIVIDADES DE USAID EN CADA OIR',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    font=dict(
        family="Arial",
        size=15,
        color="black"
    ),
    yaxis=dict(
        tickformat=',d',
        dtick=1
    )
)

# Update the text position to be inside the bars
fig.update_traces(texttemplate='%{y}', textposition='inside')

# Show the plot
fig.show()

In [ ]:
spreadsheet = gc.open('Tablas de Base de Datos RIS_1_07_2024')
worksheet = spreadsheet.worksheet('OIR_OBJETIVOS')

# Retrieve all values from the worksheet
data_1 = worksheet.get_all_values()

# Extract column titles from the first row
column_titles_1 = data_1[0]

# Extract data rows excluding the first row
data_rows_1 = data_1[1:]

# Create DataFrame with column titles
data_frame_2 = spark.createDataFrame(data_rows_1, column_titles_1)

data_frame_2.createOrReplaceTempView("OBJETIVOS")

# Show DataFrame
data_frame_2.show()

+--------+-------------+--------------------+
|  ID_OIR| ID_OBJETIVOS|           OBJETIVOS|
+--------+-------------+--------------------+
|PCF_OIR1|PCF_OIR1_OBJ1|Fortalecer las ca...|
|PCF_OIR1|PCF_OIR1_OBJ2|Generar capacidad...|
|PCF_OIR2|PCF_OIR2_OBJ1|Fortalecer la inc...|
|PCF_OIR2|PCF_OIR2_OBJ2|Fortalecimiento d...|
|PCF_OIR2|PCF_OIR2_OBJ3|Fortalecimiento d...|
|PCF_OIR3|PCF_OIR3_OBJ1|Promover la inclu...|
|PCF_OIR3|PCF_OIR3_OBJ2|Fortalecer instan...|
|PCF_OIR3|PCF_OIR3_OBJ3|Acompañar y propi...|
|PCF_OIR4|PCF_OIR4_OBJ1|Cualificar a los ...|
|PCF_OIR4|PCF_OIR4_OBJ2|Fomentar los méto...|
|BCN_OIR1|BCN_OIR1_OBJ1|Intercambiar expe...|
|BCN_OIR1|BCN_OIR1_OBJ2|Ofrecer servicios...|
|BCN_OIR1|BCN_OIR1_OBJ3|Trabajar de maner...|
|BCN_OIR1|BCN_OIR1_OBJ4|Intercambiar info...|
|BCN_OIR1|BCN_OIR1_OBJ5|Priorizar accione...|
|BCN_OIR1|BCN_OIR1_OBJ6|Hacer seguimiento...|
|BCN_OIR1|BCN_OIR1_OBJ7|Implementar inici...|
|BCN_OIR1|BCN_OIR1_OBJ8|Implementar la Es...|
|BCN_OIR3|BCN_OIR3_OBJ1|Expansión 

In [ ]:
# Filter OIR where ID_OIR contains 'PCF' or 'BCN'
data_frame_2 = spark.sql("SELECT * FROM OBJETIVOS WHERE ID_OIR LIKE '%PCF%' OR ID_OIR LIKE '%BCN%'")
data_frame_2.show()

+--------+-------------+--------------------+
|  ID_OIR| ID_OBJETIVOS|           OBJETIVOS|
+--------+-------------+--------------------+
|PCF_OIR1|PCF_OIR1_OBJ1|Fortalecer las ca...|
|PCF_OIR1|PCF_OIR1_OBJ2|Generar capacidad...|
|PCF_OIR2|PCF_OIR2_OBJ1|Fortalecer la inc...|
|PCF_OIR2|PCF_OIR2_OBJ2|Fortalecimiento d...|
|PCF_OIR2|PCF_OIR2_OBJ3|Fortalecimiento d...|
|PCF_OIR3|PCF_OIR3_OBJ1|Promover la inclu...|
|PCF_OIR3|PCF_OIR3_OBJ2|Fortalecer instan...|
|PCF_OIR3|PCF_OIR3_OBJ3|Acompañar y propi...|
|PCF_OIR4|PCF_OIR4_OBJ1|Cualificar a los ...|
|PCF_OIR4|PCF_OIR4_OBJ2|Fomentar los méto...|
|BCN_OIR1|BCN_OIR1_OBJ1|Intercambiar expe...|
|BCN_OIR1|BCN_OIR1_OBJ2|Ofrecer servicios...|
|BCN_OIR1|BCN_OIR1_OBJ3|Trabajar de maner...|
|BCN_OIR1|BCN_OIR1_OBJ4|Intercambiar info...|
|BCN_OIR1|BCN_OIR1_OBJ5|Priorizar accione...|
|BCN_OIR1|BCN_OIR1_OBJ6|Hacer seguimiento...|
|BCN_OIR1|BCN_OIR1_OBJ7|Implementar inici...|
|BCN_OIR1|BCN_OIR1_OBJ8|Implementar la Es...|
|BCN_OIR3|BCN_OIR3_OBJ1|Expansión 

In [ ]:
import pandas as pd
from google.colab import files

# Convert PySpark DataFrame to Pandas DataFrame
df_objetivos = data_frame_2.toPandas()

In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Select only the ID_REGION and NOMBRE_OIR columns
filtered_df_obj = df_objetivos[['ID_OIR', 'OBJETIVOS']]

# Create the table
fig = go.Figure(data=[go.Table(
    columnwidth=[50, 150],  # Set the width for each column
    header=dict(values=list(filtered_df_obj.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[filtered_df_obj[col] for col in filtered_df_obj.columns],
               fill_color='lavender',
               align='left'))
])

# Customize the layout
fig.update_layout(
    title={
        'text': 'OBJETIVOS DE OIRs EN PACÍFICO Y FRONTERA NARIÑENSE Y BAJO CAUCA Y NORDESTE ANTIOQUEÑO',
        'font': {'size': 12},  # Reduce the title font size
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'pad': {'b': 10}  # Reduce the padding (distance) to the table
    },
    width=1100,  # Adjust the overall width of the table
    height=1200  # Reduce the overall height of the canvas
)

# Show the plot
fig.show()